In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import numpy as np
import math
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets
from torchvision import models
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader

In [3]:
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [7]:
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5],   # 1 for greyscale channels
                                     std=[0.5])])

In [8]:
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../../data/MNIST\raw\train-images-idx3-ubyte.gz to ../../data/MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../../data/MNIST\raw\train-labels-idx1-ubyte.gz to ../../data/MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../../data/MNIST\raw\t10k-images-idx3-ubyte.gz to ../../data/MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../../data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ../../data/MNIST\raw



In [9]:
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [10]:
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

In [11]:
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

In [12]:
D = D.to(device)
G = G.to(device)

In [13]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [14]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)


def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [17]:
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

       
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
    
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
       
        g_loss = criterion(outputs, real_labels)
        
        
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
   
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
  
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))


torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

Epoch [0/200], Step [200/600], d_loss: 0.0757, g_loss: 4.7370, D(x): 0.97, D(G(z)): 0.04
Epoch [0/200], Step [400/600], d_loss: 1.1128, g_loss: 3.4064, D(x): 0.83, D(G(z)): 0.46
Epoch [0/200], Step [600/600], d_loss: 0.0633, g_loss: 4.5192, D(x): 0.98, D(G(z)): 0.04
Epoch [1/200], Step [200/600], d_loss: 0.4822, g_loss: 3.1462, D(x): 0.80, D(G(z)): 0.12
Epoch [1/200], Step [400/600], d_loss: 0.1757, g_loss: 4.7427, D(x): 0.95, D(G(z)): 0.06
Epoch [1/200], Step [600/600], d_loss: 1.2438, g_loss: 1.9564, D(x): 0.69, D(G(z)): 0.41
Epoch [2/200], Step [200/600], d_loss: 0.2244, g_loss: 3.3885, D(x): 0.92, D(G(z)): 0.09
Epoch [2/200], Step [400/600], d_loss: 1.0849, g_loss: 2.3027, D(x): 0.77, D(G(z)): 0.32
Epoch [2/200], Step [600/600], d_loss: 0.3019, g_loss: 2.7042, D(x): 0.90, D(G(z)): 0.13
Epoch [3/200], Step [200/600], d_loss: 1.7842, g_loss: 1.9497, D(x): 0.52, D(G(z)): 0.34
Epoch [3/200], Step [400/600], d_loss: 0.2692, g_loss: 3.9183, D(x): 0.90, D(G(z)): 0.08
Epoch [3/200], Step [